## Исследование опроса клиентов телекоммуникационной компании.

## Описание проекта

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
*  9-10 баллов — «cторонники» (англ. promoters);
*  7-8 баллов — «нейтралы» (англ. passives);
*  0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».
Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 
Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса.


## Основная задача

Опубликуйте дашборд на сайте Tableau Public.

Используя дашборд, ответьте на вопросы:
*  Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
*  Какие группы пользователей наиболее лояльны к сервису? Какие менее?
*  Какой общий NPS среди всех опрошенных?
*  Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

Ответы на вопросы оформите в виде презентации.


## Используемые инструменты

* Pandas
* Python
* SQLite
* Tableau

## Подключаемся к базе данных и сохраняем данные в датафрейм в pandas.

Загружаем библиотеки.

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
#Путь к БД на компьютере.
path_to_db_local = 'telecomm_csi.db'
#Путь к БД на платформе.
path_to_db_platform = '/datasets/telecomm_csi.db'
#Итоговый путь к БД.
path_to_db = None

#Если путь на компьютере ведет к БД, то он становится итоговым.
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
#Иначе: если путь на платформе ведет к БД, то он становится итоговым.
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
#Иначе выводится сообщение о том, что файл не найден.
else:
    raise Exception('Файл с базой данных SQLite не найден!')
#Если итоговый путь не пустой,
if path_to_db:
    #то создаем подключение к базе.
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
#Делаем запрос.
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'Новый'
           ELSE 'Старый'
           END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment == 1 THEN 'Женщина'
           WHEN u.gender_segment == 0 THEN 'Мужчина'
           ELSE 'z'
           END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       location.country,
       location.city,
       SUBSTR(age_segment.title, 4) AS age_segment,
       SUBSTR(traffic_segment.title, 4) AS traffic_segment,
       SUBSTR(lifetime_segment.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'Cторонник'
           WHEN u.nps_score >= 7 THEN 'Нейтрал'
           ELSE 'Критик'
           END AS nps_group
FROM user AS u
JOIN location ON u.location_id = location.location_id
JOIN age_segment ON u.age_gr_id = age_segment.age_gr_id
JOIN traffic_segment ON u.tr_gr_id = traffic_segment.tr_gr_id
JOIN lifetime_segment ON u.lt_gr_id = lifetime_segment.lt_gr_id;
"""

In [4]:
#Создаем датафрейм по данным запроса.
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Cторонник
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Cторонник
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Cторонник


В таблице 14 столбцов:

* user_id — идентификатор клиента
* lt_day — количество дней «жизни» клиента
* is_new — поле хранит информацию о том, является ли клиент новым
* age — возраст
* gender_segment — пол
* os_name — тип операционной системы
* cpe_type_name — тип устройства
* country — страна проживания
* city — город проживания
* age_segment — возрастной сегмент
* traffic_segment — сегмент по объёму потребляемого трафика
* lifetime_segment — сегмент по количеству дней «жизни»
* nps_score — оценка клиента в NPS-опросе
* nps_group — поле хранит информацию о том, к какой группе относится оценка клиента в опросе

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Вывод

* Больше всего пользователей в возрасте от 35 - 44 лет (33%), далее идет возрастная категория 25-34 года (30,8%). Замыкает рейтинг от 45 - 54 лет (18,8%).
* По половому признаку в возрасте от 35 - 44 лет пользователей женщин (17,5%), это чуть больше чем мужчин (15,4%). В категории от 25 - 34 практически равное количество - женщин 15,5%, мужчин 15,2%. А вот в категории 45 - 54 лет  разрыв между мужчинами и женщинами увеличивается - женщин снова больше (10,9%), мужчин (7,8%).
* Если смотреть соотношение пользователей между новыми и старыми, то старых пользователей ожидаемо больше - это 82,9%. Новых пользователей всего 17,1%.
* Если смотреть на то, из каких городов пользователи активнее участвовали в опросе, то картина следующая:  активнее всего принимали участие пользователи из Москвы (13,2%). Далее идет Санкт-Петербург  (5,9%). За ними Новосибирск - 5,3% и Екатеринбург - 4,6%

Всего в опросе поучаствовало 502 493 человека. Общий уровень NPS среди всех опрошенных составил 21,9%. Из графика видно, что опрашиваемые разделились на 3 группы.
Сторонники - 264 081 человек (52,6%), Критики -  154 044 человека (30,7%), Нейтралы - 84 368 человек (16,8%).

Описание сторонников:
* Возрастная группа от 66 и выше, пол: женщина. «Возраст» клиента «новый».

Описани критиков:
* Возрастная группа до 16 лет, пол: мужчина. «Возраст» клиента «старый».

Cамый низкий NPS среди сторонников в  Твери(45%), Рязани(44%), Воронеже(44%) и Тольятти(44%).

Среди критиков в Иваново(19%), Саранске(20%), Череповце(21%), Курске(21%), Новокузнецке(22%), Волжском (23%) и Архангельске(24%).


Некоторая связь между количеством участников в городах и NPS прослеживается. Например среди сторонников количество участников в этих городах не превышало 2,1%.
Среди критиков - 1%.

## Дашборды

https://public.tableau.com/app/profile/roman.ivashov/viz/NPS21/sheet14?publish=yes

## Презентация

https://cloud.mail.ru/public/VoeV/BpqCRn5Px